In [1]:
import config
# from io_utils import write_yaml_file

import os
from keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from callbacks import LogLearningRate, LogTimestamp, StopAfterTimedelta, StopWhenValLossExploding

import model
import data_provider

/home/prateek/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class Training():
    
    def __init__(self, conf=None, num_epochs=None, model_weights_path=None, min_lr=1e-7, min_delta=1e-4, explode_ratio=0.25, explode_patience=2, verbose=1):
        
        self.active_config = conf or config.Config()
        self.num_epochs = num_epochs or self.active_config.num_epochs
        self.time_limit = self.active_config.time_limit
        
        self.lr_reduce_factor = self.active_config.lr_reduce_factor
        self.lr_reduce_patience = self.active_config.lr_reduce_patience
        self.min_lr = min_lr
        self.min_delta = min_delta
        
        self.early_stopping_patience = self.active_config.early_stopping_patience
        self.model_weights_path = model_weights_path
        self.use_pre_trained_word_embeddings = self.active_config.use_pre_trained_word_embeddings
        
#         self._log_metrics_period = log_metrics_period

        self.explode_ratio = explode_ratio
        self.explode_patience = explode_patience
        
        
#         self._max_q_size = max_q_size
#         self._workers = workers
        self.verbose = verbose
        
        
        self.data_provider = data_provider.DataProvider()
        
        if self.use_pre_trained_word_embeddings:
            self.pretrained_embeddings = self.data_provider.caption_preprocessor.embedding_matrix
        else:
            self.pretrained_embeddings = None
#         self.active_config.init_vocab_size(self.data_provider.vocabulary_size)
        
        self.init_result_dir()
        self.init_callbacks()
        self.model = model.ImageCaptioningModel(vocab_size=self.data_provider.vocabulary_size, pretrained_embeddings=self.pretrained_embeddings)
        
        #TODO
#         self.write_active_config()
    
    def run(self):
        self.model.build()
        
        if self.model_weights_path:
            logging('Loading model weights from {}..'.format(
                                                    self.model_weights_path))
            self.model.load_weights(self.model_weights_path)
        
        
        self.model.keras_model.fit_generator(
                generator=self.data_provider.training_set_generator(),
                steps_per_epoch=self.data_provider.training_steps_per_epoch,
                epochs=self.num_epochs,
                validation_data=self.data_provider.validation_set_generator(),
                validation_steps=self.data_provider.validation_steps_per_epoch,
#                 max_q_size=self._max_q_size,
#                 workers=self._workers,
                callbacks=self.callbacks,
#                 verbose=self.verbose
        )
        
    def init_callbacks(self):
        log_lr = LogLearningRate()
        log_ts = LogTimestamp()
#         log_metrics = LogMetrics(self._dataset_provider,
#                                  period=self._log_metrics_period)

        CSV_FILENAME = 'metrics-log.csv'
        self.csv_filepath = self.path_from_dir(CSV_FILENAME)
        csv_logger = CSVLogger(filename=self.csv_filepath)

        CHECKPOINT_FILENAME = 'model-weights.hdf5'
        self.checkpoint_filepath = self.path_from_dir(CHECKPOINT_FILENAME)
        model_checkpoint = ModelCheckpoint(filepath=self.checkpoint_filepath,
                                           monitor='val_cider',
                                           mode='max',
                                           save_best_only=True,
                                           save_weights_only=True,
                                           period=1,
                                           verbose=self.verbose)

#         tensorboard = TensorBoard(log_dir=self._result_dir,
#                                   write_graph=False)

        reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                      mode='min',
                                      epsilon=self.min_delta,
                                      factor=self.lr_reduce_factor,
                                      patience=self.lr_reduce_patience,
                                      min_lr=self.min_lr,
                                      verbose=self.verbose)

        earling_stopping = EarlyStopping(monitor='val_loss',
                                         mode='min',
                                         min_delta=self.min_delta,
                                         patience=self.early_stopping_patience,
                                         verbose=self.verbose)

        stop_after = StopAfterTimedelta(timedelta=self.time_limit,
                                        verbose=self.verbose)

        stop_when = StopWhenValLossExploding(ratio=self.explode_ratio,
                                             patience=self.explode_patience,
                                             verbose=self.verbose)

        # TODO Add LearningRateScheduler. Is it still needed?

        self.callbacks = [log_lr,  # Must be before tensorboard
#                            log_metrics,  # Must be before model_checkpoint and
#                                          # tensorboard
                           model_checkpoint,
#                            tensorboard,  # Must be before log_ts
                           log_ts,  # Must be before csv_logger
                           csv_logger,
                           reduce_lr,  # Must be after csv_logger
                           stop_when,  # Must be the third last
                           earling_stopping,  # Must be the second last
                           stop_after]  # Must be the last
        
        
    def init_result_dir(self):
        self.results_dir = self.data_provider.training_results_dir

        CONFIG_FILENAME = 'hyperparams-config.yaml'
        config_filepath = self.path_from_dir(CONFIG_FILENAME)
        if os.path.exists(config_filepath):
            raise ValueError('Training directory already exists!')
        
    def path_from_dir(self, path):
        return os.path.join(self.results_dir, path)
  

#     def write_active_config(self):
#         CONFIG_FILENAME = 'hyperparams-config.yaml'
#         self.config_filepath = self.path_from_dir(CONFIG_FILENAME)
        
#         self.config.write_to_file(active_config, self.config_filepath)

In [3]:
train = Training()

Loading Training Data
loading annotations into memory...
Done (t=11.61s)
creating index...
index created!
loading annotations into memory...
Done (t=0.85s)
creating index...
index created!

Loading Validation Data
loading annotations into memory...
Done (t=0.43s)
creating index...
index created!
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


/home/prateek/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [4]:
train.run()

Epoch 1/50
 202/9247 [..............................] - ETA: 7:49:26 - loss: 15.8327 - acc: 0.1896

KeyboardInterrupt: 

In [ ]:
# import os

In [ ]:
# path = '/media/prateek/New Volume/Datasets/Glove.6B/glove.6B.300d.txt'

In [ ]:
# f = open(path, 'rb')